1. Get crv emissions per pool.
2. Get fees claimed per pool.
3. divide the two to get swap revenue per crv emitted per gauge.

In [1]:
# subgraph querying modules
import datetime
import calendar

In [2]:
# plotting tools
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# brownie
import os
import sys
sys.path.append("../../")

import brownie

from utils.contract_utils import init_contract
from utils.network_utils import configure_network_and_connect
from utils.eth_blocks_utils import get_block_for_timestamp


In [4]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']

if not brownie.network.is_connected():
    configure_network_and_connect(
        node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
        network_name='mainnet'
    )

# Pools and Gauges

Get dict of pools (keys) and gauge (value)

In [5]:
ethereum_registry_contract = init_contract("0x90E00ACe148ca3b23Ac1bC8C240C2a7Dd9c2d7f5")
ethereum_registry_contract.info()

  @title Curve Registry
  @author Curve.Fi



Get pools and gauges:

In [15]:
pool_count = ethereum_registry_contract.pool_count()
eth_pools_and_gauges = {}
for pool_id in range(pool_count):

    pool_addr = ethereum_registry_contract.pool_list(pool_id)
    gauge_addr = ethereum_registry_contract.get_gauges(pool_addr)
    lp_token_addr = ethereum_registry_contract.get_lp_token(pool_addr)

    # get lp_token name:
    pool_token_contract = init_contract(lp_token_addr)
    pool_token_name = pool_token_contract.name()

    eth_pools_and_gauges[pool_addr] = [gauge_addr[0][0], lp_token_addr, pool_token_name]

eth_pools_and_gauges

{'0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7': ['0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A',
  '0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490',
  'Curve.fi DAI/USDC/USDT'],
 '0xDeBF20617708857ebe4F679508E7b7863a8A8EeE': ['0xd662908ADA2Ea1916B3318327A97eB18aD588b5d',
  '0xFd2a8fA60Abd58Efe3EeE34dd494cD491dC14900',
  'Curve.fi aDAI/aUSDC/aUSDT'],
 '0xA96A65c051bF88B4095Ee1f2451C2A9d43F53Ae2': ['0x6d10ed2cF043E6fcf51A0e7b4C2Af3Fa06695707',
  '0xaA17A236F2bAdc98DDc0Cf999AbB47D47Fc0A6Cf',
  'Curve.fi ETH/aETH'],
 '0x79a8C46DeA5aDa233ABaFFD40F3A0A2B1e5A4F27': ['0x69Fb7c45726cfE2baDeE8317005d3F94bE838840',
  '0x3B3Ac5386837Dc563660FB6a0937DFAa5924333B',
  'Curve.fi yDAI/yUSDC/yUSDT/yBUSD'],
 '0xA2B47E3D5c44877cca798226B7B8118F9BFb7A56': ['0x7ca5b0a2910B33e9759DC7dDB0413949071D7575',
  '0x845838DF265Dcd2c412A1Dc9e959c7d08537f8a2',
  'Curve.fi cDAI/cUSDC'],
 '0x0Ce6a5fF5217e38315f87032CF90686C96627CAA': ['0x90Bb609649E0451E5aD952683D64BD2d1f245840',
  '0x194eBd173F6cDacE046C53eACcE9B953F28411d1',

# CRV emissions per pool:

In [7]:
crv_token_addr = "0xD533a949740bb3306d119CC777fa900bA034cd52"
crv_token = init_contract(crv_token_addr)
crv_token.info()

  @title Curve DAO Token
  @notice ERC20 with piecewise-linear mining supply.
  @author Curve Finance
  @details Based on the ERC-20 token standard as defined
           at https://eips.ethereum.org/EIPS/eip-20



CRV emissions rate per block

In [8]:
crv_token.rate() * 1e-18

7.327853447857532

get number of blocks between two timestamps:

In [9]:
block_time_start = int(
    calendar.timegm((datetime.datetime.now() - datetime.timedelta(days=7)).date().timetuple())
)
block_time_end = int(
    calendar.timegm((datetime.datetime.now()).date().timetuple())
)
crv_token.mintable_in_timeframe(block_time_start, block_time_end) * 1e-18

4431885.7652642345

get gauge weights from registry for block:

In [10]:
gauge_controller_address = "0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB"
gauge_controller = init_contract(gauge_controller_address)
gauge_controller.info()

  @title Gauge Controller
  @notice Controls liquidity gauges and the issuance of
          coins through the gauges
  @author Curve Finance



Get crv emitted for gauge

In [25]:
time_start = int(datetime.datetime.strptime("2021-06-25", "%Y-%m-%d").timestamp())
time_end = int(datetime.datetime.strptime("2021-11-04", "%Y-%m-%d").timestamp())
delta_time = int(datetime.timedelta(days=7).total_seconds())
date_range = list(range(time_start,time_end,delta_time))

In [26]:
eth_pools_and_gauges = {
    '0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7': ['0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A',
                                                   '0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490',
                                                   'Curve.fi DAI/USDC/USDT'],
    '0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1': ['0xF98450B5602fa59CC66e1379DFfB6FDDc724CfC4',
                                                   '0x4f3E8F405CF5aFC05D68142F3783bDfE13811522',
                                                   'Curve.fi USDN/3Crv'],
}

In [27]:
emissions_data = pd.DataFrame()
for timestamp in date_range:

    block_number = get_block_for_timestamp(timestamp=timestamp)

    with brownie.multicall(block_identifier=block_number):

        for pool_addr, pool_details in eth_pools_and_gauges.items():

            pool_data = {}

            gauge_addr = pool_details[0]
            lp_token_addr = pool_details[1]
            pool_name = pool_details[2]

            # get crv emissions for timestamp from the past day:
            crv_minted = crv_token.mintable_in_timeframe(
                int(timestamp - delta_time),
                timestamp
            ) * 1e-18

            # get gauge relative weight
            gauge_relative_weight = gauge_controller.gauge_relative_weight(gauge_addr) * 1e-18

            # get virtual price:
            try:
                pool = init_contract(pool_addr)
                virtual_price = pool.get_virtual_price() * 1e-18
                total_coins = 0
                for coin_index in range(5):
                    try:
                        total_coins += pool.balances(coin_index)
                    except:
                        break
            except:
                virtual_price = 0

            # get total supply of lp_token:
            lp_token = init_contract(lp_token_addr)
            total_supply_lp_token = lp_token.totalSupply() * 1e-18

            # pool data
            pool_data['timestamp'] = timestamp
            pool_data['block_number'] = block_number
            pool_data['pool_name'] = pool_name
            pool_data['pool_addr'] = pool_addr
            pool_data['lp_token_addr'] = lp_token_addr
            pool_data['gauge_addr'] = gauge_addr
            pool_data['gauge_relative_weight'] = gauge_relative_weight
            pool_data['lp_token_virtual_price'] = virtual_price
            pool_data['total_supply_lp_token'] = total_supply_lp_token
            pool_data['crv_minted_past_1d'] = crv_minted
            pool_data['crv_to_gauge'] = gauge_relative_weight * crv_minted

            pool_data = pd.DataFrame(
                data=pool_data,
                index=[pd.to_datetime(pool_data['timestamp'], unit='s')]
            )

            # join df
            emissions_data = pd.concat([emissions_data, pool_data])


In [38]:
pool = init_contract(pool_addr)
pool.balances(1)

122000463404310427981430066

Let's look at the data:

In [28]:
emissions_data

,timestamp,block_number,pool_name,pool_addr,lp_token_addr,gauge_addr,gauge_relative_weight,lp_token_virtual_price,total_supply_lp_token,crv_minted_past_1d,crv_to_gauge
2021-06-24 22:00:00,1624572000,12699406,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.030511,1.017823,1.764436e+09,5.270430e+06,160805.062896
2021-06-24 22:00:00,1624572000,12699406,Curve.fi USDN/3Crv,0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1,0x4f3E8F405CF5aFC05D68142F3783bDfE13811522,0xF98450B5602fa59CC66e1379DFfB6FDDc724CfC4,0.134197,1.030940,5.627110e+08,5.270430e+06,707277.038120
2021-07-01 22:00:00,1625176800,12744302,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.029063,1.017912,1.731845e+09,5.270430e+06,153176.715303
2021-07-01 22:00:00,1625176800,12744302,Curve.fi USDN/3Crv,0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1,0x4f3E8F405CF5aFC05D68142F3783bDfE13811522,0xF98450B5602fa59CC66e1379DFfB6FDDc724CfC4,0.128277,1.031422,5.663490e+08,5.270430e+06,676075.223567
2021-07-08 22:00:00,1625781600,12789307,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.027046,1.017976,1.733671e+09,5.270430e+06,142546.091241
2021-07-08 22:00:00,1625781600,12789307,Curve.fi USDN/3Crv,0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1,0x4f3E8F405CF5aFC05D68142F3783bDfE13811522,0xF98450B5602fa59CC66e1379DFfB6FDDc724CfC4,0.130238,1.031807,5.925849e+08,5.270430e+06,686410.078913
2021-07-15 22:00:00,1626386400,12834098,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.028084,1.018046,1.536708e+09,5.270430e+06,148015.566716
2021-07-15 22:00:00,1626386400,12834098,Curve.fi USDN/3Crv,0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1,0x4f3E8F405CF5aFC05D68142F3783bDfE13811522,0xF98450B5602fa59CC66e1379DFfB6FDDc724CfC4,0.127589,1.032265,5.819403e+08,5.270430e+06,672450.339207
2021-07-22 22:00:00,1626991200,12878749,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.030080,1.018120,1.467889e+09,5.270430e+06,158536.154743
2021-07-22 22:00:00,1626991200,12878749,Curve.fi USDN/3Crv,0x0f9cb53Ebe405d49A0bbdBD291A65Ff571bC83e1,0x4f3E8F405CF5aFC05D68142F3783bDfE13811522,0xF98450B5602fa59CC66e1379DFfB6FDDc724CfC4,0.136702,1.032666,5.488982e+08,5.270430e+06,720477.588054


# swap fees to liquidity providers per pool per week:

In [29]:
emissions_and_revenue = pd.DataFrame()
for pool_addr in emissions_data.pool_addr.unique():

    pool_data = emissions_data[emissions_data.pool_addr == pool_addr]
    pool_data = pool_data.sort_values(by="block_number", ascending=True)
    pool_data['virtual_price_diff'] = pool_data.lp_token_virtual_price.diff()

    emissions_and_revenue = pd.concat([emissions_and_revenue, pool_data])

In [30]:
emissions_and_revenue

,timestamp,block_number,pool_name,pool_addr,lp_token_addr,gauge_addr,gauge_relative_weight,lp_token_virtual_price,total_supply_lp_token,crv_minted_past_1d,crv_to_gauge,virtual_price_diff
2021-06-24 22:00:00,1624572000,12699406,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.030511,1.017823,1.764436e+09,5.270430e+06,160805.062896,NaN
2021-07-01 22:00:00,1625176800,12744302,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.029063,1.017912,1.731845e+09,5.270430e+06,153176.715303,0.000089
2021-07-08 22:00:00,1625781600,12789307,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.027046,1.017976,1.733671e+09,5.270430e+06,142546.091241,0.000065
2021-07-15 22:00:00,1626386400,12834098,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.028084,1.018046,1.536708e+09,5.270430e+06,148015.566716,0.000070
2021-07-22 22:00:00,1626991200,12878749,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.030080,1.018120,1.467889e+09,5.270430e+06,158536.154743,0.000074
2021-07-29 22:00:00,1627596000,12923235,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.030036,1.018220,1.457812e+09,5.270430e+06,158301.362771,0.000099
2021-08-05 22:00:00,1628200800,12967525,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.025019,1.018337,1.439797e+09,5.270430e+06,131859.565557,0.000118
2021-08-12 22:00:00,1628805600,13012938,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.025891,1.018451,1.467864e+09,5.270430e+06,136459.087412,0.000114
2021-08-19 22:00:00,1629410400,13058324,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.023078,1.018550,1.428284e+09,4.553131e+06,105076.565343,0.000099
2021-08-26 22:00:00,1630015200,13103596,Curve.fi DAI/USDC/USDT,0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7,0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490,0xbFcF63294aD7105dEa65aA58F8AE5BE2D9d0952A,0.021880,1.018636,1.379284e+09,4.431886e+06,96969.213000,0.000086


save data:

In [31]:
t_start = datetime.datetime.fromtimestamp(time_start).strftime('%Y%m%d')
t_end = datetime.datetime.fromtimestamp(time_end).strftime('%Y%m%d')

In [32]:
f_name = f"./emissions_and_revenue_" \
         f"{t_start}_" \
         f"{t_end}.csv"
emissions_and_revenue.to_csv(f_name)
f_name

'./emissions_and_revenue_20210625_20211104.csv'